
## Purpose

Test out clusterProfiler

## Load libraries


In [ ]:
# Bioconductor
library(clusterProfiler)
#library(paxtoolsr)

# CRAN
library(wordcloud)
library(tm)
library(igraph)
library(magrittr)
library(dplyr)
library(jsonlite)

source("readGmt.R")
source("toCytoscape.R")


## Load Gene Sets
### Bader EnrichmentMap


In [ ]:
lowerBound <- 3
upperBound <- 100

lovGmt <- readGmt("Human_GOBP_AllPathways_no_GO_iea_February_01_2017_symbol.gmt")
xCnts <- unlist(lapply(lovGmt, length))
idx <- which(xCnts > lowerBound & xCnts < upperBound)
which.max(xCnts[idx])
length(lovGmt[[1435]])

orgNames <- names(lovGmt)

smNames <- NULL
for(i in 1:length(lovGmt)) {
  t1 <- names(lovGmt)[i]
  t2 <- strsplit(t1, "%")
  smNames <- c(smNames, t2[[1]][1])
}


# Convert GMT to clusterProfiler format
NOTE: This is slow


In [ ]:
pc <- lovGmt[idx]
emGmt <- data.frame(ont=character(0), gene=character(0), db=character(0), id=character(0))

maxValue <- length(pc)
pb <- txtProgressBar(min=1, max=maxValue, style=3)

for(i in 1:length(names(pc))) {
  setTxtProgressBar(pb, i)

  t1 <- names(pc)[i]
  t2 <- strsplit(t1, "%")
  ont <- t2[[1]][1]
  db <- t2[[1]][2]
  id <- t2[[1]][3]

  emGmt <- rbind(emGmt, data.frame(ont=ont, gene=pc[[i]], db=db, id=id))
}

saveRDS(emGmt, "emGmt.rds")


## Load GMT


In [ ]:
emGmt <- readRDS("emGmt.rds")

gmt <- emGmt


## Load Example Data


In [ ]:
genes <- geneSetPathwayAnalysis::geneSets["DDR (HR)"]
genes <- genes$DDR
genes <- sample(genes, 15)
genes


## Run Enrichment Analyses


In [ ]:
# Hypergeometric
egmt <- enricher(genes, TERM2GENE=gmt, qvalueCutoff=0.1)
head(egmt)

barplot(egmt, showCategory=8)
dotplot(egmt, showCategory=8)
g <- enrichMap(egmt, n=10, vertex.label.font=0.001)

j1 <- toCytoscape(g)
write(j1, "enrich_del.json")


## Get communities (modules)


In [ ]:
x <- enrichMap(egmt, n=nrow(egmt), vertex.label.font=0.05)
ebc <- cluster_edge_betweenness(x)
ebc <- cluster_fast_greedy(x)
e1 <- membership(ebc)

e1Df <- data.frame(name=names(e1), module=as.numeric(e1), stringsAsFactors = FALSE)

# Largest module
curModule <- as.numeric(names(sort(table(unname(e1)),decreasing=TRUE)[1]))
curModule <- 3

e2 <- names(which(e1 == curModule))

ebcGenes <- NULL
for(i in 1:length(e2)) {
  pattern <- gsub("([.|()\\^{}+$*?]|\\[|\\])", "\\\\\\1", e2[i])
  pattern <- paste0("^", pattern, "%")

  # Grab the first geneset if multiple
  idx <- which(grepl(pattern, names(lovGmt)))[1]

  genesInPathway <- genes[genes %in% lovGmt[[idx]]]

  ebcGenes <- c(ebcGenes, genesInPathway)
}

sort(table(ebcGenes), decreasing=TRUE) / length(e2)
#ebcGenes <- unique(ebcGenes)

# Recurrent words in module and frequency
#wordcloud(corpus, max.words = 25, min.freq=3, random.order = FALSE)
#tf1

# Genes in module
ebcGenes

# Get the gene set name that has the most overlap with the user input
idxM1 <- which(egmt$ID %in% e2)
idxM2 <- which.max(egmt$Count[idxM1])
selectedPathway <- egmt$ID[idxM1][idxM2]